In [1]:
import pandas as pd
import numpy as np
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
audio_files='./Music_Data/genres_original'
file_data=pd.read_csv('./Music_Data/features_30_sec.csv')
file_data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [3]:
extracted_features=[]
for index_num,row in file_data.iterrows():
    try:
        final_class_labels=row["label"]
        file_name = os.path.join(os.path.abspath(audio_files), final_class_labels+'/',str(row["filename"]))    
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        scaled_features = np.mean(features.T,axis=0)
        extracted_features.append([scaled_features,final_class_labels])
    except Exception as e:
        print(f"Error: {e}")
        continue

/home/ben/miniconda3/envs/Flask_env/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


Error: 


In [4]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','genre'])
extracted_features_df.head()

,feature,genre
0,"[-113.59882, 121.57067, -19.162262, 42.36394, ...",blues
1,"[-207.52383, 123.98514, 8.947019, 35.86715, 2....",blues
2,"[-90.757164, 140.44087, -29.084547, 31.686693,...",blues
3,"[-199.57513, 150.0861, 5.663404, 26.855278, 1....",blues
4,"[-160.35417, 126.20948, -35.581394, 22.139256,...",blues


In [5]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['genre'].tolist())

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.18,random_state=0)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [9]:
model=Sequential()
model.add(Dense(1024,input_shape=(40,), activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(512, activation="sigmoid"))
model.add(Dropout(0.3))
model.add(Dense(256, activation="sigmoid"))
model.add(Dropout(0.3))
model.add(Dense(128, activation="sigmoid"))
model.add(Dropout(0.3))
model.add(Dense(64, activation="sigmoid"))
model.add(Dropout(0.3))
model.add(Dense(32, activation="sigmoid"))
model.add(Dropout(0.3))


###final layer
num_labels=y.shape[1]
model.add(Dense(num_labels, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              41984     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3

In [10]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [11]:
model.fit(X_train, y_train, batch_size=40, epochs=350, validation_data=(X_test, y_test), verbose=1)

Epoch 1/350
21/21 [==============================] - 3s 79ms/step - loss: 2.4681 - accuracy: 0.0925 - val_loss: 2.3281 - val_accuracy: 0.0667
Epoch 2/350
21/21 [==============================] - 1s 27ms/step - loss: 2.3473 - accuracy: 0.1052 - val_loss: 2.3137 - val_accuracy: 0.0556
Epoch 3/350
21/21 [==============================] - 1s 28ms/step - loss: 2.3430 - accuracy: 0.0879 - val_loss: 2.3093 - val_accuracy: 0.1389
Epoch 4/350
21/21 [==============================] - 0s 22ms/step - loss: 2.3196 - accuracy: 0.1081 - val_loss: 2.3015 - val_accuracy: 0.1444
Epoch 5/350
21/21 [==============================] - 0s 22ms/step - loss: 2.3223 - accuracy: 0.1041 - val_loss: 2.2807 - val_accuracy: 0.1389
Epoch 6/350
21/21 [==============================] - 1s 33ms/step - loss: 2.2892 - accuracy: 0.1511 - val_loss: 2.2601 - val_accuracy: 0.1333
Epoch 7/350
21/21 [==============================] - 0s 22ms/step - loss: 2.2664 - accuracy: 0.1504 - val_loss: 2.2246 - val_accuracy: 0.2000
Epoch 

In [12]:
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=128)
print("Test loss is: ", test_loss)
print("The best accuracy is: ", test_acc*100)

2/2 [==============================] - 0s 21ms/step - loss: 2.4190 - accuracy: 0.6333
Test loss is:  2.419037103652954
The best accuracy is:  63.333332538604736


Current best accuracy: 63.33 (sigmoid activation, 82/18 split, 40 batch, 350 epoch)


In [13]:
model.save('./music_model')

INFO:tensorflow:Assets written to: ./music_model/assets


In [14]:
filename="./Trivium - Rise Above The Tides.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
predicted_label=model.predict_classes(mfccs_scaled_features)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class

[6]


/home/ben/miniconda3/envs/Flask_env/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array(['metal'], dtype='<U9')